In [73]:
import torch
from torch import empty, zeros
import math
import copy

In [74]:
torch.manual_seed(1)

In [75]:
class Module(object):
    def __init__(self):
        pass
        
    def forward(self, *input_):
        raise NotImplementedError
        
    def backward(self, *gradwrtoutput):
        raise NotImplementedError
        
    def param(self):
        return []
    
    def update(self, lr):
        pass
    
    def zero_grad(self):
        pass
    
    def init_params(self, xavier_init, xavier_gain):
        pass


In [76]:
class Linear(Module):
    def __init__(self, N_in, N_out, xavier_init=False, xavier_gain=1):
        Module.__init__(self)
        self.N_in = N_in
        self.N_out = N_out
        
        self.init_params(xavier_init, xavier_gain)
        
        self.gradW = zeros(self.W.shape)
        self.gradb = zeros(self.b.shape)
        
        self.m_weights = zeros(self.gradW.shape)
        self.m_bias = zeros(self.gradb.shape)
        
        self.v_weights = zeros(self.gradW.shape)
        self.v_bias = zeros(self.gradb.shape)
    
    def init_params(self, xavier_init=True, xavier_gain=1):
        if xavier_init:
            xavier_std = xavier_gain * math.sqrt(2.0 / (self.N_in + self.N_out))
        else:
            xavier_std = 1
            
        self.W = empty((self.N_in, self.N_out)).normal_(0, xavier_std)
        self.b = empty((1, self.N_out)).normal_(0, xavier_std)
    
    def forward(self, *input_):
        # out = W * input + b
        x = input_[0].clone()
        
        self.x = x
        
        return self.x.mm(self.W) + self.b
        
    def backward(self, *gradwrtoutput):
        # grad_w += input * x^(l-1).t()
        # grad_b += input
        # out = w.t() * input
        # input = grad of activation function, i.e. dl/ds^(l)
        # x^(l-1) = input of the forward pass
        input_ = gradwrtoutput[0].clone()
        
        self.gradW += self.x.t().mm(input_)
        self.gradb += input_.sum(0)
        
        return input_.mm(self.W.t())
        
    def param(self):
        return [(self.W, self.gradW, self.m_weights, self.v_weights), (self.b, self.gradb, self.m_bias, self.v_bias)]
    
    def update(self, lr):
        self.W.sub_(lr * self.gradW)
        self.b.sub_(lr * self.gradb)
    
    def zero_grad(self):
        self.gradW = zeros(self.W.shape)
        self.gradb = zeros(self.b.shape)

In [77]:
class ReLU(Module):
    def __init__(self, ):
        Module.__init__(self)
        
    def forward(self, *input_):
        s = input_[0].clone()
        self.s = s
        
        s[s < 0] = 0.
        
        return s
        
    def backward(self, *gradwrtoutput):
        # out = f'(s^(l)) * input
        # s^(l) = input of forward pass
        # input = grad of next layer
        input_ = gradwrtoutput[0].clone()
        
        out = self.s.clone()
        out[out > 0] = 1
        out[out < 0] = 0
        
        
        return out.mul(input_)

In [78]:
class LeakyReLU(Module):
    def __init__(self, alpha=0.01):
        Module.__init__(self)
        self.alpha = alpha
        
    def forward(self, *input_):
        s = input_[0].clone()
        self.s = s
        
        s[s < 0] = self.alpha * s[s < 0]
        
        return s
        
    def backward(self, *gradwrtoutput):
        # out = f'(s^(l)) * input
        # s^(l) = input of forward pass
        # input = grad of next layer
        input_ = gradwrtoutput[0].clone()
        
        out = self.s.clone()
        out[out > 0] = 1
        out[out < 0] = self.alpha
        
        
        return out.mul(input_)

In [79]:
class Tanh(Module):
    def __init__(self):
        Module.__init__(self)
        
    def forward(self, *input_):
        s = input_[0].clone()
        self.s = s
        
        return s.tanh()
        
    def backward(self, *gradwrtoutput):
        # out = f'(s^(l)) * input
        # s^(l) = input of forward pass
        # input = grad of next layer
        input_ = gradwrtoutput[0].clone()
        
        out = self.s.clone()
        out = 1 - out.tanh().pow(2)
        
        return out.mul(input_)

In [80]:
class Sigmoid(Module):
    def __init__(self):
        Module.__init__(self)
        
    def forward(self, *input_):
        s = input_[0].clone()
        self.s = s
        
        return s.sigmoid()
        
    def backward(self, *gradwrtoutput):
        # out = f'(s^(l)) * input
        # s^(l) = input of forward pass
        # input = grad of next layer
        input_ = gradwrtoutput[0].clone()
        
        out = self.s.clone()
        out = out.sigmoid() * (1 - out.sigmoid())
        
        return out.mul(input_)

In [81]:
class Sequential(Module):
    def __init__(self, *modules, xavier_init=None, xavier_gain=1):
        Module.__init__(self)
        self.modules = modules
        self.xavier_init = xavier_init
        if xavier_init is not None:
            for module in self.modules:
                module.init_params(xavier_init, xavier_gain)
        
    def forward(self, *input_):
        x = input_[0].clone()
        
        for m in self.modules:
            x = m.forward(x)
        
        return x
        
    def backward(self, *gradwrtoutput):
        x = gradwrtoutput[0].clone()
        
        for i, m in enumerate(reversed(self.modules)):
            #print("{} : {}".format(i, x))
            x = m.backward(x)
        
    def param(self):
        params = []
        
        for m in self.modules:
            for param in m.param():
                params.append(param)
        
        return params

    def update(self, lr):
        for m in self.modules:
            m.update(lr)
    
    def zero_grad(self):
        for m in self.modules:
            m.zero_grad()

In [82]:
"""class Loss: 
    ...
"""
class LossMSE(Module):
    def __init__(self):
        Module.__init__(self)
        
    def forward(self, y, target):
        # out = e^2
        # e = (y - f(x))
        self.y = y.clone()
        target_onehot = zeros((target.shape[0], 2)) 
        self.target = target_onehot.scatter_(1, target.view(-1, 1), 1)
        
        self.e = (self.y - self.target)
        self.n = self.y.size(0)
        
        
        return self.e.pow(2).sum()
        
    def backward(self):
        # out = 2 * e
        
        return 2 * self.e # / self.n        

In [83]:
class LossCrossEntropy(Module):
    def __init__(self):
        Module.__init__(self)
        
    def forward(self, y, target):
        self.y = y.clone()
        self.target = target.clone()
        sm = torch.softmax(self.y, dim=1)
        likelihood = -torch.log(torch.clamp(sm[range(target.size(0)), target], min=1e-3, max=None))
        return likelihood.mean()
        
    def backward(self):
        sm = torch.softmax(self.y, dim=1)
        target_onehot = zeros((self.target.shape[0], 2)) 
        target_onehot = target_onehot.scatter_(1, self.target.view(-1, 1), 1)
        return sm - target_onehot

In [97]:
class Optimizer:
    def __init__(self, model, nb_epochs, mini_batch_size, lr, criterion):
        self.model = model
        self.nb_epochs = nb_epochs
        self.lr = lr
        self.mini_batch_size = mini_batch_size
        self.criterion = criterion
    
    def train(self, train_input, train_target, verbose=True):
        for e in range(self.nb_epochs):
            sum_loss = 0.
        
            for b in range(0, train_input.size(0), self.mini_batch_size):
                self.model.zero_grad()
                
                output = self.model.forward(train_input.narrow(0, b, self.mini_batch_size))
                loss = self.criterion.forward(output, train_target.narrow(0, b, self.mini_batch_size))
            
                sum_loss += loss
            
                l_grad = self.criterion.backward()
                self.model.backward(l_grad)
                self.step()
                
            if verbose:
                print("{} iteration: loss={}".format(e, sum_loss))
        return self.model
    
    def step(self):
        raise NotImplementedError

class SGD(Optimizer):
    def __init__(self, model, nb_epochs = 50, mini_batch_size=1, lr=1e-4, criterion=LossMSE()):
        Optimizer.__init__(self, model, nb_epochs, mini_batch_size, lr, criterion)
    
    def step(self):
        self.model.update(self.lr)

In [85]:
class Adam(Optimizer):
    def __init__(self, model, nb_epochs = 50, mini_batch_size=1, lr=1e-3, criterion=LossMSE(), 
                 b1=0.9, b2=0.999, epsilon=1e-8):
        Optimizer.__init__(self, model, nb_epochs, mini_batch_size, lr, criterion)
        
        self.b1 = b1
        self.b2 = b2
        self.epsilon = epsilon
        self.t = 0
    
    def step(self):
        self.t += 1
        
        for (param, grad, m, v) in self.model.param():
            g = grad.clone()
            
            m = self.b1 * m + (1 - self.b1) * g
            v = self.b2 * v + (1 - self.b2) * g.pow(2)
            
            m_biasc = m / (1 - self.b1 ** self.t)
            v_biasc = v / (1 - self.b2 ** self.t)
            
            param.sub_(self.lr * m_biasc/(v_biasc.sqrt() + self.epsilon))

In [124]:
class Evaluator:
    def __init__(self, model):
        self.model = model
        
    def cross_validate(self, k, values):
        pass
    
    def test(self, test_input, test_target):
        num_samples = test_input.size(0)
        prediction = self.model.forward(test_input)
        predicted_class = torch.argmax(prediction, axis=1)
        accuracy = sum(predicted_class == test_target).float() / num_samples
        return accuracy

In [140]:
class EvaluatorSGD(Evaluator):
    def __init__(self, model, nb_epochs = 50, mini_batch_size=1, lr=1e-4, criterion=LossMSE()):
        Evaluator.__init__(self, model)
        self.optimizer = SGD(model=model, nb_epochs=nb_epochs, mini_batch_size=mini_batch_size, 
                             lr=lr, criterion=criterion)
        
    def cross_validate(self, k=5, values={"lr": [1e-5, 1e-4, 1e-3, 1e-2]}):
        train_datasets = []
        test_datasets = []
        for i in range(k):
            train_datasets.append(generate_disc_set(1000))
            test_datasets.append(generate_disc_set(1000))
        
        if "lr" not in values:
            raise ValueError("Expected learning rate values to cross-validate...")
        
        possible_lrs = values["lr"]
        
        score_means = []
        score_vars = []
        for lr in possible_lrs:
            print("Validating (lr={})".format(lr))
            scores = []
            optimizer.lr = lr
            
            for (train_input, train_target), (test_input, test_target) in zip(train_datasets, test_datasets):
                optimizer.model = copy.deepcopy(self.model) 
                
                self.model = optimizer.train(train_input, train_target, verbose=False)
                accuracy = self.test(test_input, test_target)
                scores.append(accuracy)
            
            scores = torch.FloatTensor(scores)
            score_means.append(torch.mean(scores).item())
            score_vars.append(torch.std(scores).item())
        best_score = {}
        
        i = max(enumerate(score_means), key=lambda x: x[1])[0]
        
        best_score["lr"] = possible_lrs[i]
        best_score["mean"] = score_means[i]
        best_score["std"] = score_vars[i]
        
        return dict(zip(possible_lrs, zip(score_means, score_vars))), best_score

In [151]:
class EvaluatorAdam(Evaluator):
    def __init__(self, model, nb_epochs = 50, mini_batch_size=1, lr=1e-4, criterion=LossMSE(),
                b1=0.9, b2=0.999, epsilon=1e-8):
        Evaluator.__init__(self, model)
        self.optimizer = Adam(model=model, nb_epochs=nb_epochs, mini_batch_size=mini_batch_size, 
                              lr=lr, criterion=criterion, b1=b1, b2=b2, epsilon=epsilon)
        
    def cross_validate(self, k=5, values={"lr": [1e-5, 1e-4, 1e-3, 1e-2], "b1": [0.9], 
                                          "b2": [0.999], "epsilon": [1e-8]}):
        train_datasets = []
        test_datasets = []
        for i in range(k):
            train_datasets.append(generate_disc_set(1000))
            test_datasets.append(generate_disc_set(1000))
        
        if "lr" not in values or "b1" not in values or "b1" not in values or "epsilon" not in values:
            raise ValueError("Expected learning rate values to cross-validate...")
            
        if "b1" not in values:
            raise ValueError("Expected b1 values to cross-validate...")

        if "b2" not in values:
            raise ValueError("Expected b2 values to cross-validate...")

        if "epsilon" not in values:
            raise ValueError("Expected epsilon values to cross-validate...")
        
        lrs = values["lr"]
        b1s = values["b1"]
        b2s = values["b2"]
        epsilons = values["epsilon"]
        param_grid = [(lr, b1, b2, epsilon)
                        for lr in lrs
                        for b1 in b1s
                        for b2 in b2s
                        for epsilon in epsilons]
        
        score_means = []
        score_vars = []
        for (lr, b1, b2, epsilon) in param_grid:
            print("Validating (lr={}, b1={}, b2={}, epsilon={})...".format(lr, b1, b2, epsilon))
            scores = []
            
            optimizer.lr = lr
            optimizer.b1 = b1
            optimizer.b2 = b2
            optimizer.epsilon = epsilon
                
            for (train_input, train_target), (test_input, test_target) in zip(train_datasets, test_datasets):
                optimizer.model = copy.deepcopy(self.model) 
                self.model = optimizer.train(train_input, train_target, verbose=False)
                accuracy = self.test(test_input, test_target)
                scores.append(accuracy)
            
            scores = torch.FloatTensor(scores)
            score_means.append(torch.mean(scores).item())
            score_vars.append(torch.std(scores).item())
        best_score = {}
        
        i = max(enumerate(score_means), key=lambda x: x[1])[0]
        
        best_score["lr"] = param_grid[i][0]
        best_score["b1"] = param_grid[i][1]
        best_score["b2"] = param_grid[i][2]
        best_score["epsilon"] = param_grid[i][3]
        best_score["mean"] = score_means[i]
        best_score["std"] = score_vars[i]
        
        return dict(zip(param_grid, zip(score_means, score_vars))), best_score

In [142]:
def generate_disc_set(nb):
    input = empty(nb, 2).uniform_(-1, 1)
    target = input.pow(2).sum(1).sub(2 / math.pi).sign().add(1).div(2).long()
    return input, target

train_input, train_target = generate_disc_set(1000)
test_input, test_target = generate_disc_set(1000)

In [152]:
# model = Sequential(Linear(2, 25), ReLU(),
#                    Linear(25, 25), ReLU(),
#                    Linear(25, 25), ReLU(),
#                    Linear(25, 2))
# model = Sequential(Linear(2, 25), Tanh(),
#                    Linear(25, 25), Tanh(),
#                    Linear(25, 25), Tanh(),
#                    Linear(25, 2))
model = Sequential(Linear(2, 25), Sigmoid(),
                   Linear(25, 25), Sigmoid(),
                   Linear(25, 25), Sigmoid(),
                   Linear(25, 2))

evaluator = EvaluatorAdam(model, mini_batch_size=100)

In [153]:
values = {"lr": [1e-6, 1e-5, 1e-4, 1e-3, 1e-2], "b1": [0.9, 0.8], 
          "b2": [0.999, 0.888], "epsilon": [1e-8, 1e-7, 1e-6]}
evaluator.cross_validate(values=values)

Validating (lr=1e-06, b1=0.9, b2=0.999, epsilon=1e-08)...
Validating (lr=1e-06, b1=0.9, b2=0.999, epsilon=1e-07)...
Validating (lr=1e-06, b1=0.9, b2=0.999, epsilon=1e-06)...
Validating (lr=1e-06, b1=0.9, b2=0.888, epsilon=1e-08)...
Validating (lr=1e-06, b1=0.9, b2=0.888, epsilon=1e-07)...
Validating (lr=1e-06, b1=0.9, b2=0.888, epsilon=1e-06)...
Validating (lr=1e-06, b1=0.8, b2=0.999, epsilon=1e-08)...
Validating (lr=1e-06, b1=0.8, b2=0.999, epsilon=1e-07)...
Validating (lr=1e-06, b1=0.8, b2=0.999, epsilon=1e-06)...
Validating (lr=1e-06, b1=0.8, b2=0.888, epsilon=1e-08)...
Validating (lr=1e-06, b1=0.8, b2=0.888, epsilon=1e-07)...
Validating (lr=1e-06, b1=0.8, b2=0.888, epsilon=1e-06)...
Validating (lr=1e-05, b1=0.9, b2=0.999, epsilon=1e-08)...
Validating (lr=1e-05, b1=0.9, b2=0.999, epsilon=1e-07)...
Validating (lr=1e-05, b1=0.9, b2=0.999, epsilon=1e-06)...
Validating (lr=1e-05, b1=0.9, b2=0.888, epsilon=1e-08)...
Validating (lr=1e-05, b1=0.9, b2=0.888, epsilon=1e-07)...
Validating (lr

({(1e-06, 0.9, 0.999, 1e-08): (0.39239999651908875, 0.033731285482645035),
  (1e-06, 0.9, 0.999, 1e-07): (0.4447999894618988, 0.0225876085460186),
  (1e-06, 0.9, 0.999, 1e-06): (0.4668000340461731, 0.02459065429866314),
  (1e-06, 0.9, 0.888, 1e-08): (0.5016000270843506, 0.02810337394475937),
  (1e-06, 0.9, 0.888, 1e-07): (0.5331999659538269, 0.017584076151251793),
  (1e-06, 0.9, 0.888, 1e-06): (0.5635999441146851, 0.01939845085144043),
  (1e-06, 0.8, 0.999, 1e-08): (0.5867999792098999, 0.015738470479846),
  (1e-06, 0.8, 0.999, 1e-07): (0.61080002784729, 0.015139340423047543),
  (1e-06, 0.8, 0.999, 1e-06): (0.6306000351905823, 0.01728583686053753),
  (1e-06, 0.8, 0.888, 1e-08): (0.647599995136261, 0.01327779982239008),
  (1e-06, 0.8, 0.888, 1e-07): (0.6620000004768372, 0.012980761006474495),
  (1e-06, 0.8, 0.888, 1e-06): (0.6808000206947327, 0.012794535607099533),
  (1e-05, 0.9, 0.999, 1e-08): (0.8255999684333801, 0.062151435762643814),
  (1e-05, 0.9, 0.999, 1e-07): (0.9010000228881836,

In [95]:
best_lr = 1e-3
model = Sequential(Linear(2, 25), ReLU(),
                   Linear(25, 25), Tanh(),
                   Linear(25, 25), Sigmoid(),
                   Linear(25, 2), xavier_init=False)
optimizer = Adam(model, lr=best_lr, mini_batch_size=100, criterion=LossCrossEntropy())
model = optimizer.train(train_input, train_target)
evaluator.model = model
evaluator.test(test_input, test_target)

0 iteration: loss=8.486946105957031
1 iteration: loss=7.5531768798828125
2 iteration: loss=6.805228233337402
3 iteration: loss=6.143959045410156
4 iteration: loss=5.591228485107422
5 iteration: loss=5.16232967376709
6 iteration: loss=4.821000576019287
7 iteration: loss=4.5264739990234375
8 iteration: loss=4.251053810119629
9 iteration: loss=3.9840495586395264
10 iteration: loss=3.7258665561676025
11 iteration: loss=3.4747889041900635
12 iteration: loss=3.2319607734680176
13 iteration: loss=2.998076915740967
14 iteration: loss=2.7800989151000977
15 iteration: loss=2.576714277267456
16 iteration: loss=2.3910794258117676
17 iteration: loss=2.222191572189331
18 iteration: loss=2.070324659347534
19 iteration: loss=1.9336928129196167
20 iteration: loss=1.8113350868225098
21 iteration: loss=1.7018442153930664
22 iteration: loss=1.6020692586898804
23 iteration: loss=1.51361882686615
24 iteration: loss=1.4326446056365967
25 iteration: loss=1.3594814538955688
26 iteration: loss=1.293104290962219

tensor(0.9890)